<a href="https://colab.research.google.com/github/p-disha/ShopUNow-Agent/blob/main/ShopUNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####################################


##Tech Stack

LangChain / langchain_community-	Provides VectorStores (FAISS), Document abstraction, Embeddings, and Retrieval.
FAISS (vectorstore)-	For embedding storage & similarity search (RAG).
Sentence-Transformers embeddings-	To convert document chunks into embedding vectors.
**pdfminer.six + pytesseract + PIL**-	Extract text from PDFs, images (OCR) and markdown/text files — for building corpus.
Markdownify	Convert markdown files to plain text.
LangGraph (StateGraph etc.)-	The agent orchestration framework: state + nodes + transitions.
Pydantic-	For structured schemas of state and tool inputs (validation, typing).
LLM backends- OpenAI, Gemini (if available)	For synthesis / general LLM responses.

**Parses different document types (text, csv, pdf, image) into a corpus.**

**Chunks documents into manageable pieces using RecursiveCharacterTextSplitter.**

**Builds a FAISS index, persists it.**

**Sets up intent routing + tools for order status, returns, tickets.**

**Handles RAG retrieval + LLM synthesis with system prompt.**

**Passes retriever via RunnableConfig/configurable, avoiding earlier bug.**

**Good structure using StateGraph, Pydantic state schemas.**

# Task
Modify the code to use Gemini as the primary LLM and fallback to OpenAI, and add tools for order status, returns, and tickets.

## Integrate llms (gemini and openai)

### Subtask:
Modify the agent to use a language model for answer synthesis, with Gemini as the primary and OpenAI as a fallback.


**Reasoning**:
Define the `get_chat_model` function to handle primary (Gemini) and fallback (OpenAI) LLM initialization and then update the `synthesis_node` to use this LLM for answer generation based on context and user input.



**Reasoning**:
The previous command failed because `langchain_google_genai` was not installed. Install the missing package and try the imports and function definition again.



In [36]:
# =========================
# Cell A: Setup and Vector Store + LLM
# =========================

# Install required packages (include Google-GenAI integration if using Gemini)
!pip install -qU langchain_community faiss-cpu langchain_openai langchain-google-genai pydantic typing_extensions vaderSentiment langgraph

!pip install -qU flask flask-cors pyngrok


# Imports
import os
from typing import List, Dict, Any, Optional, Literal
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage

# Make sure API keys are set
try:
    from google.colab import userdata
    key = userdata.get("OPENAI_API_KEY")
    if key:
        os.environ["OPENAI_API_KEY"] = key
    gem_key = userdata.get("GEMINI_API_KEY")
    if gem_key:
        os.environ["GOOGLE_API_KEY"] = gem_key
except Exception:
    pass

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GOOGLE_API_KEY")  # for Gemini

assert OPENAI_API_KEY or GEMINI_API_KEY, "Please set OPENAI_API_KEY or GEMINI_API_KEY in environment variables."

# Initialize embeddings (use OpenAI embeddings; you can use Gemini embeddings if you want and have the key)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Sample documents / FAQ data with department metadata
# You should expand these to 10-15 QA per department later
faq_docs = [
    Document(page_content="Support hours are 9 AM–9 PM IST, Monday to Saturday", metadata={"department": "Customer Support"}),
    Document(page_content="How to contact support email or phone", metadata={"department": "Customer Support"}),
    Document(page_content="Return window is 10 days from delivery", metadata={"department": "Orders & Returns"}),
    Document(page_content="How can I initiate a return process", metadata={"department": "Orders & Returns"}),
    Document(page_content="We accept UPI, credit cards, wallets, and COD", metadata={"department": "Payments & Billing"}),
    Document(page_content="How to apply coupon at checkout", metadata={"department": "Payments & Billing"})
]

# Build the FAISS vector store
import faiss

# Compute embedding dimension
dim = len(embeddings.embed_query("hello world"))  # length of vector

index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

ids = [f"doc{i+1}" for i in range(len(faq_docs))]
vector_store.add_documents(documents=faq_docs, ids=ids)

# Initialize LLM (Chat model)
def get_chat_model():
    if GEMINI_API_KEY:
        try:
            print("Using Gemini LLM")
            return ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
        except Exception as e:
            print("Gemini init failed:", e)
    if OPENAI_API_KEY:
        try:
            print("Using OpenAI model")
            return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        except Exception as e:
            print("OpenAI init failed:", e)
    # fallback mock model
    class _Mock:
        def invoke(self, messages: List[Any]):
            last_user = None
            for m in reversed(messages):
                if isinstance(m, HumanMessage):
                    last_user = m
                    break
            return type("Obj", (), {"content": "[MOCK] " + (last_user.content if last_user else "")})
    print("Using mock LLM fallback")
    return _Mock()

LLM = get_chat_model()

# Optionally define system policy / prompt template
SYSTEM_POLICY = (
    "You are ShopUNow Assistant. Be concise and accurate. Use the internal knowledge base when possible. "
    "Cite sources. If you cannot answer, ask a clarifying question."
)

print("Cell A setup complete: vector_store and LLM are initialized.")


Using Gemini LLM
Cell A setup complete: vector_store and LLM are initialized.


In [37]:
# =========================
# Cell A.1 - Load FAQ Dataset & Build Vector Store
# =========================
import json
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
import faiss

# ---- Step 1: Load JSONL file ----
jsonl_path = "/content/shopunow_faqs.jsonl"
docs = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:   # skip empty lines
            continue
        try:
            record = json.loads(line)
            docs.append(
                Document(
                    page_content=record["answer"],
                    metadata={
                        "department": record.get("department", "unknown"),
                        "question": record.get("question", "")
                    }
                )
            )
        except json.JSONDecodeError as e:
            print(f"⚠️ Skipping bad line: {line[:80]}... | Error: {e}")

print(f"Loaded {len(docs)} FAQs from {jsonl_path}")

# ---- Step 2: Build FAISS Vector Store ----
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

ids = [f"doc{i+1}" for i in range(len(docs))]
vector_store.add_documents(documents=docs, ids=ids)

print(f"✅ Vector store built with {len(docs)} documents across {len(set(d.metadata['department'] for d in docs))} departments")


Loaded 75 FAQs from /content/shopunow_faqs.jsonl
✅ Vector store built with 75 documents across 5 departments


In [38]:
# =========================
# Cell B - Agent Definition with JSONL Vector Store
# =========================
import json
import faiss
from typing import Optional, List, Dict, Any, Literal
from typing_extensions import Annotated
from operator import add
from pydantic import BaseModel, Field
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# ---- Load FAQ dataset from JSONL ----
jsonl_path = "/content/shopunow_faqs.jsonl"
docs = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        docs.append(
            Document(
                page_content=record["answer"],
                metadata={"department": record["department"], "question": record["question"]}
            )
        )

print(f"Loaded {len(docs)} FAQs from {jsonl_path}")

# ---- Build FAISS vector store ----
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

ids = [f"doc{i+1}" for i in range(len(docs))]
vector_store.add_documents(documents=docs, ids=ids)

print(f"✅ Vector store ready with {len(docs)} documents across {len(set(d.metadata['department'] for d in docs))} departments")

# ---- Sentiment + Dept Classifier ----
_sentiment_analyzer = SentimentIntensityAnalyzer()

def detect_sentiment(text: str) -> Literal["negative","neutral","positive"]:
    if not text:
        return "neutral"
    scores = _sentiment_analyzer.polarity_scores(text)
    c = scores.get("compound", 0.0)
    if c <= -0.3:
        return "negative"
    if c >= 0.3:
        return "positive"
    return "neutral"

def classify_department(text: str) -> Optional[str]:
    t = (text or "").lower()
    if any(kw in t for kw in ["order status", "track order", "where is my order", "order tracking"]):
        return "Orders & Returns"
    if any(kw in t for kw in ["return", "refund", "replace", "exchange"]):
        return "Orders & Returns"
    if any(kw in t for kw in ["payment", "upi", "card", "wallet", "cod", "invoice", "coupon"]):
        return "Payments & Billing"
    if any(kw in t for kw in ["support", "contact", "help", "issue", "complaint"]):
        return "Customer Support"
    if any(kw in t for kw in ["password", "vpn", "access", "onboarding", "hardware", "software"]):
        return "HR & IT Helpdesk"
    return None

# ---- Agent State ----
class AgentState(BaseModel):
    user_input: str
    department: Optional[str] = None
    sentiment: Optional[Literal["negative","neutral","positive"]] = None
    tools_used: Annotated[List[str], add] = Field(default_factory=list)
    retrieved: Annotated[List[Dict[str, Any]], add] = Field(default_factory=list)
    intent: Optional[Literal["rag","order_status","return_create","ticket","human_escalation","unknown"]] = None
    answer: Optional[str] = None

# ---- Routing ----
def route_intent(state: AgentState) -> Dict[str, Any]:
    q = state.user_input
    low = (q or "").lower()
    sentiment = detect_sentiment(q)
    dept = classify_department(q)

    if sentiment == "negative":
        intent = "human_escalation"
    elif any(kw in low for kw in ["order status", "track order", "where is my order", "order tracking"]):
        intent = "order_status"
    elif any(kw in low for kw in ["return", "refund", "replace", "exchange"]):
        intent = "return_create"
    elif any(kw in low for kw in ["ticket", "helpdesk", "support issue", "complaint", "problem"]):
        intent = "ticket"
    else:
        intent = "rag"

    print(f"[route_intent] input={q!r} -> intent={intent}, dept={dept}, sentiment={sentiment}")
    return {"intent": intent, "department": dept, "sentiment": sentiment}

# ---- Tool Node ----
def _postfilter_by_dept(docs: List[Any], dept: Optional[str]) -> List[Any]:
    if not docs:
        return []
    if dept is None:
        return docs
    filtered = [d for d in docs if (d.metadata or {}).get("department") == dept]
    return filtered or docs

def tool_node(state: AgentState) -> Dict[str, Any]:
    intent = state.intent
    q = state.user_input or ""
    dept = state.department
    print(f"[tool_node] intent={intent}, dept={dept}, input={q!r}")

    if intent == "order_status":
        return {"answer": "Your order is being processed and will be shipped soon.", "tools_used": ["order_status_tool"]}

    if intent == "return_create":
        return {"answer": "Return initiated. You will receive pickup and label details via email.", "tools_used": ["return_create_tool"]}

    if intent == "ticket":
        return {"answer": "A support ticket has been created. Someone will get back to you shortly.", "tools_used": ["ticket_tool"]}

    if intent == "human_escalation":
        return {"answer": "I’m sorry for the inconvenience. Escalating to human support — someone will reach out to you soon.", "tools_used": ["escalation"]}

    if intent == "rag":
        results = vector_store.similarity_search(q, k=5)
        results = _postfilter_by_dept(results, dept)
        if results:
            top = results[0]
            dept_meta = (top.metadata or {}).get("department", "unknown")
            return {
                "answer": f"{top.page_content} (Dept: {dept_meta})",
                "tools_used": ["rag_retrieval"],
                "retrieved": [{"content": top.page_content, "source": dept_meta}]
            }
        else:
            return {"answer": "Sorry, no relevant info found in our knowledge base.", "tools_used": ["rag_retrieval"]}

    return {"answer": "Could you please rephrase your request?", "tools_used": ["fallback"]}

# ---- Synthesis Node ----
def synthesis_node(state: AgentState) -> Dict[str, Any]:
    return {}

# ---- Build Graph ----
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(AgentState)
graph.add_node("route", route_intent)
graph.add_node("tool", tool_node)
graph.add_node("synth", synthesis_node)

graph.add_edge(START, "route")
graph.add_edge("route", "tool")
graph.add_edge("tool", "synth")
graph.add_edge("synth", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# ---- Ask Function ----
def ask(q: str, thread_id: Optional[str] = None) -> str:
    if thread_id is None:
        import uuid
        thread_id = f"thread_{uuid.uuid4().hex}"
    out = app.invoke({"user_input": q},
                     config={"configurable": {"thread_id": thread_id}})
    return out.get("answer", "No answer generated.")


Loaded 75 FAQs from /content/shopunow_faqs.jsonl
✅ Vector store ready with 75 documents across 5 departments


In [39]:
# =========================
# Cell B.1 - Testing
# =========================

import uuid

for q in [
    "What are your support hours?",
    "Tell me order status for order id ORD-1234",
    "I want a return because the product is wrong",
    "My password reset isn't working, this is frustrating",
    "I submitted a complaint about a support issue",
    "How do I pay with UPI?",
    "Please replace my shirt size",
    "I need help,"  # ambiguous but should go ticket/support
]:
    # Create fresh thread_id each iteration
    thread_id = f"thread_{uuid.uuid4().hex}"
    state = AgentState(user_input=q)
    info = route_intent(state)
    print(f"{q} → intent: {info['intent']} | dept: {info['department']} | sentiment: {info['sentiment']}")
    print("Answer:", ask(q, thread_id=thread_id))
    print("----")


[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, sentiment=positive
What are your support hours? → intent: rag | dept: Customer Support | sentiment: positive
[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, sentiment=positive
[tool_node] intent=rag, dept=Customer Support, input='What are your support hours?'
Answer: Currently, support is available from 9 AM to 9 PM IST, Monday through Saturday. Emergency queries can be raised via the support portal. (Dept: Customer Support)
----
[route_intent] input='Tell me order status for order id ORD-1234' -> intent=order_status, dept=Orders & Returns, sentiment=neutral
Tell me order status for order id ORD-1234 → intent: order_status | dept: Orders & Returns | sentiment: neutral
[route_intent] input='Tell me order status for order id ORD-1234' -> intent=order_status, dept=Orders & Returns, sentiment=neutral
[tool_node] intent=order_status, dept=Orders & Returns, inpu

In [42]:
# =========================
# Cell C - Flask API with ngrok (collision-safe, deep debug, auto-reuse)
# =========================
import os, sys, traceback, threading, uuid
from flask import Flask, request, jsonify
from flask_cors import CORS

# --------- Flask Setup (separate var from LangGraph 'app') ---------
flask_app = Flask(__name__)
CORS(flask_app)

def _debug(msg):
    print(msg, flush=True)

# --------- Agent Caller ---------
def call_agent(query: str) -> str:
    if "ask" in globals() and callable(globals()["ask"]):
        _debug(f"[AGENT] Using ask()")
        return globals()["ask"](query)

    for name in ["agent_app", "graph_app", "app"]:
        obj = globals().get(name)
        if hasattr(obj, "invoke"):
            _debug(f"[AGENT] Using graph '{name}'.invoke()")
            cfg = {"configurable": {"thread_id": f"api-{uuid.uuid4().hex}"}}
            out = obj.invoke({"user_input": query}, config=cfg)
            return out.get("answer", "No answer generated.")
    raise RuntimeError("No agent available. Run Cell B first.")

# --------- Routes ---------
@flask_app.route("/ask", methods=["POST", "GET"])
def ask_api():
    try:
        _debug("\n[API] ▶️ Received /ask")
        if request.method == "POST":
            if not request.is_json:
                return jsonify({"error": "Content-Type must be application/json"}), 400
            data = request.get_json(force=True, silent=True) or {}
            query = (data.get("query") or "").strip()
        else:
            query = (request.args.get("query") or "").strip()

        if not query:
            return jsonify({"error": "Empty query"}), 400

        answer = call_agent(query)
        return jsonify({"query": query, "answer": answer})
    except Exception as e:
        traceback.print_exc(file=sys.stdout)
        return jsonify({"error": "Internal server error", "details": str(e)}), 500

@flask_app.route("/", methods=["GET"])
def home():
    return jsonify({"status": "ok", "message": "ShopUNow Agent API is running!"})

# --------- Run Flask (dynamic port, avoids collisions) ---------
from werkzeug.serving import make_server
import socket

def find_free_port(default=5000):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

PORT = find_free_port(5000)
_debug(f"▶️ Starting Flask server on port {PORT}...")

def run_flask():
    try:
        flask_app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)
    except Exception as e:
        _debug(f"❌ Flask crashed: {e}")
        traceback.print_exc(file=sys.stdout)

threading.Thread(target=run_flask, daemon=True).start()

# --------- ngrok setup ---------
try:
    from pyngrok import ngrok
except ImportError:
    _debug("[NGROK] Installing pyngrok...")
    import subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=True)
    from pyngrok import ngrok

def _get_secret(name):
    try:
        from google.colab import userdata
        return userdata.get(name)
    except Exception:
        return os.getenv(name)

NGROK_AUTH_TOKEN = _get_secret("NGROK_AUTH_TOKEN")
if NGROK_AUTH_TOKEN:
    _debug("🔑 Setting ngrok auth token")
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Kill old tunnels before opening a new one
ngrok.kill()

try:
    _debug(f"🌐 Starting ngrok tunnel on :{PORT} ...")
    tunnel = ngrok.connect(PORT)
    public_url = getattr(tunnel, "public_url", str(tunnel))
    _debug(f"🚀 Public API URL: {public_url}")
    print("\nTest with:")
    print(f'curl -X POST "{public_url}/ask" -H "Content-Type: application/json" -d "{{\\"query\\": \\"What are your support hours?\\"}}"')
except Exception as e:
    _debug(f"❌ ngrok connection failed: {e}")
    traceback.print_exc(file=sys.stdout)


▶️ Starting Flask server on port 60179...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:60179
 * Running on http://172.28.0.12:60179
INFO:werkzeug:Press CTRL+C to quit


🔑 Setting ngrok auth token
🌐 Starting ngrok tunnel on :60179 ...
🚀 Public API URL: https://31a863ccbd3a.ngrok-free.app

Test with:
curl -X POST "https://31a863ccbd3a.ngrok-free.app/ask" -H "Content-Type: application/json" -d "{\"query\": \"What are your support hours?\"}"


In [44]:
# =========================
# Cell D - Streamlit Frontend (Mandatory Keys + LangSmith optional)
# =========================
import subprocess, threading
!pip install -q streamlit requests pyngrok

with open("app_frontend.py", "w") as f:
    f.write("""
import streamlit as st
import requests

st.set_page_config(page_title="ShopUNow Agent", page_icon="🛍️", layout="centered")
st.title("🛍️ ShopUNow AI Assistant")

# --- Sidebar: Configuration ---
st.sidebar.header("🔑 Configuration (Required)")

api_url = st.sidebar.text_input("Flask API URL (ngrok/public)", value="http://127.0.0.1:5000/ask")
openai_key = st.sidebar.text_input("OpenAI API Key", type="password")
gemini_key = st.sidebar.text_input("Gemini API Key", type="password")
ngrok_token = st.sidebar.text_input("ngrok Auth Token", type="password")

st.sidebar.header("Optional")
langsmith_key = st.sidebar.text_input("LangSmith Key (optional)", type="password")

# ---- Validation ----
errors = []
if not api_url.strip():
    errors.append("❌ Flask API URL is required.")
if not (openai_key.strip() or gemini_key.strip()):
    errors.append("❌ At least one model key (OpenAI or Gemini) is required.")
if not ngrok_token.strip():
    errors.append("❌ ngrok Auth Token is required.")

if errors:
    for e in errors:
        st.sidebar.error(e)
    st.stop()
else:
    st.sidebar.success("✅ All required configuration set")

# Store secrets
st.session_state.secrets = {
    "API_URL": api_url.strip(),
    "OPENAI_API_KEY": openai_key.strip(),
    "GEMINI_API_KEY": gemini_key.strip(),
    "NGROK_AUTH_TOKEN": ngrok_token.strip(),
    "LANGSMITH_KEY": langsmith_key.strip(),
}

# --- Chat Section ---
st.subheader("💬 Chat with ShopUNow Agent")

if "chat" not in st.session_state:
    st.session_state.chat = []

query = st.text_input("Type your question here:")

if st.button("Ask"):
    if query.strip():
        st.session_state.chat.append(("🧑 You", query))
        try:
            resp = requests.post(st.session_state.secrets["API_URL"], json={"query": query}, timeout=20)
            if resp.status_code == 200:
                data = resp.json()
                answer = data.get("answer", "⚠️ No answer returned")
            else:
                answer = f"⚠️ Error {resp.status_code}: {resp.text}"
        except Exception as e:
            answer = f"⚠️ Request failed: {e}"

        st.session_state.chat.append(("🤖 Agent", answer))

# --- Display chat history ---
for sender, msg in st.session_state.chat:
    st.markdown(f"**{sender}:** {msg}")
""")

# ---- Run Streamlit in background ----
def run_streamlit():
    subprocess.run(
        ["streamlit", "run", "app_frontend.py", "--server.port", "8501", "--server.headless", "true"]
    )

threading.Thread(target=run_streamlit, daemon=True).start()

# ---- ngrok tunnel for frontend ----
from pyngrok import ngrok
print("🌐 Starting ngrok tunnel for Streamlit...")
frontend_url = ngrok.connect(8501)
print("🚀 Streamlit Frontend URL:", frontend_url)


🌐 Starting ngrok tunnel for Streamlit...
🚀 Streamlit Frontend URL: NgrokTunnel: "https://df18f55c7b61.ngrok-free.app" -> "http://localhost:8501"
